<a href="https://colab.research.google.com/github/Amari-Lawal/Data-Science-Projects-/blob/main/NLP_Practice_Attempt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenize and Remove stop words
Sequence into list of indices
Pad Sequences
Vectorise



In [1]:
# Upload a kaggle.json api key by going into settings of a kaggle account
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"stinner999palace","key":"473e7cf39093d8fb6cf2f7dd909f5633"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ # Makes the directory to store the kaggle api key to be able to access the dataset
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d team-ai/spam-text-message-classification # Downloads dataset from kaggle

  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 28.4MB/s]


In [4]:
!ls

kaggle.json  sample_data  spam-text-message-classification.zip


In [5]:
import zipfile
zip_ref = zipfile.ZipFile("spam-text-message-classification.zip", "r") # Unzips the zip file
zip_ref.extractall()
zip_ref.close()

In [6]:
DATA_URL = "/content/SPAM text message 20170820 - Data.csv"
import pandas as pd
df = pd.read_csv(DATA_URL) # Loads in the 
display(df)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
from sklearn.model_selection import train_test_split
X = df["Message"]
df_dum = pd.get_dummies(df["Category"])
ylabels = df_dum['spam'] # Set values as dummy variables
X_train, X_test, y_train, y_test = train_test_split(X,ylabels,test_size=0.3)

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)

In [11]:
print(tok)

In [12]:
tok.fit_on_texts(X_train)

In [15]:
sequences = tok.texts_to_sequences(X_train)

In [49]:
X_train_sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [31]:
len(sequences_matrix)

3900

In [45]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import *

In [124]:
def Lstm():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)  #vocab, vector space ;output of the Embedding layer is a 2D vector
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
    

In [125]:
model = Lstm()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [127]:
#,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]
model.fit(X_train_sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
25/25 [==============================] - 5s 199ms/step - loss: 0.0069 - accuracy: 0.1330 - val_loss: 0.0639 - val_accuracy: 0.1269
Epoch 2/10
25/25 [==============================] - 5s 196ms/step - loss: 0.0062 - accuracy: 0.1330 - val_loss: 0.0660 - val_accuracy: 0.1269
Epoch 3/10
25/25 [==============================] - 5s 199ms/step - loss: 0.0054 - accuracy: 0.1330 - val_loss: 0.0686 - val_accuracy: 0.1269
Epoch 4/10
25/25 [==============================] - 5s 200ms/step - loss: 0.0044 - accuracy: 0.1330 - val_loss: 0.0781 - val_accuracy: 0.1269
Epoch 5/10
25/25 [==============================] - 5s 201ms/step - loss: 0.0038 - accuracy: 0.1330 - val_loss: 0.0755 - val_accuracy: 0.1269
Epoch 6/10
25/25 [==============================] - 5s 199ms/step - loss: 0.0038 - accuracy: 0.1330 - val_loss: 0.0864 - val_accuracy: 0.1269
Epoch 7/10
25/25 [==============================] - 5s 200ms/step - loss: 0.0046 - accuracy: 0.1330 - val_loss: 0.0756 - val_accuracy: 0.1269
Epoch 

In [128]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [129]:
accr = model.evaluate(test_sequences_matrix,y_test)

53/53 [==============================] - 1s 22ms/step - loss: 0.0939 - accuracy: 0.1394


In [130]:
predicted = model.predict(test_sequences_matrix)

In [131]:
predicted = pd.DataFrame(predicted,columns=["Probability"])

In [132]:
display(predicted)

,Probability
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1667,1.0
1668,1.0
1669,1.0
1670,1.0


In [133]:
predicted_class = np.argmax(predicted.loc[:,"Probability"])

In [134]:
predicted["Class"] = predicted["Probability"].apply(lambda x: np.argmax(x))

In [135]:
print(pd.unique(predicted.loc[:,"Class"]))

[0]


In [136]:
display(predicted)

,Probability,Class
0,1.0,0
1,1.0,0
2,1.0,0
3,1.0,0
4,1.0,0
...,...,...
1667,1.0,0
1668,1.0,0
1669,1.0,0
1670,1.0,0


In [137]:
for i in range(0,len(X_test)):
  if predicted.iloc[i,1] == 1:
    print(f'Spam: {X_test.iloc[i]}')
  elif predicted.iloc[i,1] == 0:
    print(f'Not Spam: {X_test.iloc[i]}')

Not Spam: Oh and by the way you do have more food in your fridge! Want to go out for a meal tonight?
Not Spam: URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid12hrs only
Not Spam: Best msg: It's hard to be with a person, when u know that one more step foward will make u fall in love.. &amp; One step back can ruin ur friendship.. good night:-) ...
Not Spam: Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed £1000 cash or £5000 prize!
Not Spam: By the way, i've put a skip right outside the front of the house so you can see which house it is. Just pull up before it.
Not Spam: I dun believe u. I thk u told him.
Not Spam: Good morning, im suffering from fever and dysentry ..will not be able to come to office today.
Not Spam: Much better now thanks lol
Not Spam: AH POOR BABY!HOPE URFEELING BETTERSN LUV! PROBTHAT OVERDOSE OF WO

In [99]:
print(predicted_class)

1


In [98]:
print(predicted_class.shape)

()


In [77]:
print(predicted_class)

IndexError: ignored

In [73]:
import numpy as np
#print(np.argmax(predicted[1]))

0


In [66]:
text=["You are awarded a Nikon Digital Camera. Call now"]
text2 = ["Claim 900 pounds today "]

In [67]:
sen = tok.texts_to_sequences(text2)
text_matrix = sequence.pad_sequences(sen,maxlen=max_len)
model.predict(text_matrix)

array([[0.05510807]], dtype=float32)